In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [28]:
companies_from_csv = pd.read_csv('companies.csv')
companies_list = companies_from_csv['company'].tolist()
print(len(companies_list))
print(companies_list)

1045
['Walmart', 'Exxon Mobil', 'Chevron', 'Berkshire Hathaway', 'Apple', 'General Motors', 'Phillips 66', 'General Electric', 'Ford Motor', 'CVS Health', 'McKesson', 'AT&T', 'Valero Energy', 'UnitedHealth Group', 'Verizon', 'AmerisourceBergen', 'Fannie Mae', 'Costco', 'HP', 'Kroger', 'JP Morgan', 'Bank of America Corp.', 'IBM', 'Marathon Petroleum', 'Cardinal', 'Boeing', 'Citigroup', 'Amazon', 'Wells Fargo', 'Microsoft', 'Procter & Gamble | P&G', 'Home Depot', 'Archer Daniels Midland', 'Walgreens', 'Target', 'Johnson & Johnson | J&J', 'Anthem', 'MetLife', 'Google', 'State Farm Insurance Cos.', 'Freddie Mac', 'Comcast', 'PepsiCo', 'United Technologies', 'AIG', 'UPS', 'Dow Chemical', 'Aetna', "Lowe's", 'ConocoPhillips', 'Intel', 'Energy Transfer Equity', 'Caterpillar', 'Prudential Financial', 'Pfizer', 'Disney', 'Humana', 'Enterprise Products Partners', 'Cisco Systems', 'Sysco', 'Ingram Micro', 'Coca-Cola', 'Lockheed Martin', 'FedEx', 'Johnson Controls', 'Plains GP Holdings', 'World Fue

In [29]:
companies_df = pd.read_csv('companies.csv')
companies_df.head(20)

,company,location,industry
0,Walmart,"Bentonville, AR",General Merchandisers
1,Exxon Mobil,"Irving, TX",Petroleum Refining
2,Chevron,"San Ramon, CA",Petroleum Refining
3,Berkshire Hathaway,"Omaha, NE",Insurance: Property and Casualty (Stock)
4,Apple,"Cupertino, CA","Computers, Office Equipment"
5,General Motors,"Detroit, MI",Motor Vehicles and Parts
6,Phillips 66,"Houston, TX",Petroleum Refining
7,General Electric,"Fairfield, CT",Diversified Financials
8,Ford Motor,"Dearborn, MI",Motor Vehicles and Parts
9,CVS Health,"Woonsocket, RI",Food and Drug Stores


In [4]:
# Extract

page = requests.get('https://edition.cnn.com/business/tech').text
soup = BeautifulSoup(page,'html.parser')
section_element = soup.find('section', class_='layout__wrapper layout-no-rail__wrapper')
span_elements = section_element.find_all('span', class_='container__headline-text')
links = soup.find_all('a', class_='container__link container__link--type-article container_lead-plus-headlines-with-images__link')
more_links = soup.find_all('a', class_='container__link container__link--type-video container_lead-plus-headlines__link')
even_more_links = soup.find_all('a', class_='container__link container__link--type-article container_lead-plus-headlines__link')

links_list = []
for link in links:
    links_list.append(str(link['href']))

for link in more_links:
    links_list.append(str(link['href']))

for link in even_more_links:
    links_list.append(str(link['href']))

headlines = []
for headline in span_elements:
    headline = str(headline.contents)
    headline = headline[2:]
    headline = headline[:-2]
    headlines.append(headline)

headlines = set(headlines)              # To remove duplicates (may occur)

companies_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, companies_list)) + r')\b', flags=re.IGNORECASE)

matching_company_words = [              # Extract words from headlines that match entire company names
    [
        word for word in re.findall(r'\b\w+\b', headline)
        if companies_pattern.fullmatch(word) and word.lower() not in {'', ''}  # Exclude certain words
    ]
    for headline in headlines
]

filtered_headlines = [                  # Filter headlines that include matching_company_words
    headline for headline, company_words in zip(headlines, matching_company_words) if any(company_words)
]

matching_company_words = [item for item in matching_company_words if item]


df = pd.DataFrame(columns=["Company", "Headline"])      # Creation of dataframe using the  matching_company_words and filtered_headlines lists
df["Company"] = matching_company_words
df["Company"] = df["Company"].astype(str).str[1:-1]
df["Company"] = df["Company"].str.replace("'", '')
df["Headline"] = filtered_headlines

df.head(20)


,Company,Headline
0,OpenAI,505 OpenAI employees threaten to quit and call...
1,OpenAI,Sam Altman is back at OpenAI … with a guest badge
2,X,It’s ‘pathological’ for brands to advertise on...
3,X,Elon Musk’s X sues media watchdog Media Matter...
4,OpenAI,"OpenAI unveils latest AI model, customizable G..."
5,OpenAI,500+ OpenAI employees threaten to quit over CE...
6,"Facebook, Instagram",Facebook and Instagram users in Europe can now...
7,"Microsoft, OpenAI",Microsoft CEO doesn’t dismiss possibility of S...
8,OpenAI,OpenAI drama continues: Sam Altman may be mull...
9,X,Ad execs encourage X CEO Linda Yaccarino to qu...


In [5]:
## TO OBTAIN EACH HEADLINE'S ARTICLE

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

search_links = []

service = Service(executable_path='/usr/local/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(service=service, options=options)

for headline in filtered_headlines:         # Queries CNN page for news that match the headline
    headline = headline.replace(' ', '+')
    search_url = 'https://edition.cnn.com/search?q=' + headline + '&sort=relevance&from=0&size=10&page=1&types=all&section='
    print(search_url)

    driver.get(search_url)
    time.sleep(10)
    updated_page_content = driver.page_source
    cnn_search_soup = BeautifulSoup(updated_page_content,'html.parser')
    links_element = cnn_search_soup.find('a', class_='container__link container__link--type-NewsArticle container_list-images-with-description__link')
    search_links.append(links_element['href'])
    #print('link: ' + str(links_element['href']) + '\n')

driver.quit()
    

https://edition.cnn.com/search?q=505+OpenAI+employees+threaten+to+quit+and+call+on+the+board+to+resign+over+Sam+Altman’s+firing&sort=relevance&from=0&size=10&page=1&types=all&section=
https://edition.cnn.com/search?q=Sam+Altman+is+back+at+OpenAI+…+with+a+guest+badge&sort=relevance&from=0&size=10&page=1&types=all&section=
https://edition.cnn.com/search?q=It’s+‘pathological’+for+brands+to+advertise+on+Elon+Musk’s+X,+Yale+professor+says&sort=relevance&from=0&size=10&page=1&types=all&section=
https://edition.cnn.com/search?q=Elon+Musk’s+X+sues+media+watchdog+Media+Matters+over+report+on+pro-Nazi+content+on+the+social+media+site&sort=relevance&from=0&size=10&page=1&types=all&section=
https://edition.cnn.com/search?q=OpenAI+unveils+latest+AI+model,+customizable+GPTs+and+digital+store&sort=relevance&from=0&size=10&page=1&types=all&section=
https://edition.cnn.com/search?q=500++OpenAI+employees+threaten+to+quit+over+CEO+firing&sort=relevance&from=0&size=10&page=1&types=all&section=
https://edi

In [6]:
### Extract article from each link

articles_list = []

for link in search_links:       # for each page
    page = requests.get(link).text
    soup = BeautifulSoup(page,'html.parser')
    paragraph_elements = soup.find_all('p', class_='paragraph inline-placeholder')
    full_article_string = ''
    for article in paragraph_elements:
        content = str(article.contents)
        content = clean_text = re.sub(r'<.*?>', '', content)
        replacements = [("n     ", ''), ("['\ ", ''), ("n  ']", '')]
        for char, replacement in replacements:
            if char in content:
                content = content.replace(char, replacement)
        content = str(content)
        print(content)
        full_article_string = full_article_string + str(content)
    articles_list.append(full_article_string)
    #print("--------------------XXXXXXXX----------XXXXXXXX--------------------")

print(articles_list)


Hundreds of OpenAI staffers are calling for the resignation of the ChatGPT company’s board, and threatening to quit themselves, after a tumultuous weekend that began with the surprise ouster of CEO Sam Altman and ended with Altman being hired by Microsoft.\
In a letter obtained by CNN, the more than 500 employees accused the OpenAI board of mishandling Altman’s firing, failing to provide sufficient evidence for claims that Altman had not been candid with the board, and “negotiating in bad faith” with the company’s executive leadership.\
“Your actions have made it obvious that you are incapable of overseeing OpenAI,” wrote the employees. “We are unable to work for or with people that lack competence, judgement and care for our mission and employees.”\
The employees also warned that they would “imminently” follow Altman to Microsoft unless the board resigns and reinstates Altman and Greg Brockman, the former OpenAI president who was also removed by the board on Friday.\
Among the signato

In [7]:
### TO SUMMARIZE TEXT TO ONLY IMPORTANT TOPICS

import openai
from openai._client import OpenAI
import os

client = OpenAI(
  api_key = 'XXXXXXXXXXXXXXXX',
)

def get_response(prompt): 
    completions = client.completions.create(
        model = "text-davinci-003",
        prompt = prompt,
        stream = False,
        max_tokens = 1024,
        n = 1,
        stop = None,
        temperature = 0.5,
    )
    message = completions.choices[0].text
    return message

In [16]:
articles_summary_list = []
for article in articles_list:
    summary = get_response("Please summarize the following text to 40 words, and determine whether this is positive or negative news: " + article)
    summary = summary.replace("\n", "")
    articles_summary_list.append(summary)
    print(summary)
    time.sleep(21)

df["Summary"] = articles_summary_list

Hundreds of OpenAI staffers are calling for the board's resignation and threatening to quit unless Sam Altman and Greg Brockman are reinstated. Mira Murati and Ilya Sutskever, along with other senior employees, are signatories of the letter. Microsoft has assured OpenAI employees of positions. Negative news.
Positive news: Sam Altman is back at OpenAI and may return as CEO, with negotiations ongoing and conditions set. Microsoft, OpenAI's biggest investor, may also get a board seat.
Negative: X confidently claims to have industry-leading brand safety tools, yet advertisements for major companies and organizations appear on the profile account of VDARE, an openly racist, white supremacist outlet. Companies have expressed concern and have suspended advertisements, yet X has taken little action.
CCDH's Thursday filing in the Northern District Court of California claims X Corp's August lawsuit is "riddled with legal deficiencies" and attempts to punish CCDH for their First Amendment protec

In [18]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')  # Download the VADER lexicon

sid = SentimentIntensityAnalyzer()

article_sentiment_list = []
for article in articles_list:
    sentiment = sid.polarity_scores(article)
    article_sentiment_list.append(sentiment['compound'])

article_sentiment_description = []
for sentiment in article_sentiment_list:
    if sentiment >= 0.05:
        article_sentiment_description.append('Good News')
        sentiment = 'Positive'
    elif sentiment <= -0.05:
        article_sentiment_description.append('Bad News')
        sentiment = 'Negative'
    else:
        article_sentiment_description.append('Neutral')
        sentiment = 'Neutral'

df['Article Sentiment'] = article_sentiment_list
df['Article Sentiment Description'] = article_sentiment_description

summary_sentiment_list = []
for summary in articles_summary_list:
    sentiment = sid.polarity_scores(summary)
    summary_sentiment_list.append(sentiment['compound'])

summary_sentiment_description = []
for sentiment in summary_sentiment_list:
    if sentiment >= 0.05:
        summary_sentiment_description.append('Good News')
        sentiment = 'Positive'
    elif sentiment <= -0.05:
        summary_sentiment_description.append('Bad News')
        sentiment = 'Negative'
    else:
        summary_sentiment_description.append('Neutral')
        sentiment = 'Neutral'

df['Summary Sentiment'] = summary_sentiment_list
df['Summary Sentiment Description'] = summary_sentiment_description

df.head(20)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Mina/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Company,Headline,Summary,Article Sentiment,Article Sentiment Description,Summary Sentiment,Summary Sentiment Description
0,OpenAI,505 OpenAI employees threaten to quit and call...,Hundreds of OpenAI staffers are calling for th...,-0.7044,Bad News,-0.7783,Bad News
1,OpenAI,Sam Altman is back at OpenAI … with a guest badge,Positive news: Sam Altman is back at OpenAI an...,0.9497,Good News,0.5574,Good News
2,X,It’s ‘pathological’ for brands to advertise on...,Negative: X confidently claims to have industr...,-0.9363,Bad News,-0.6597,Bad News
3,X,Elon Musk’s X sues media watchdog Media Matter...,CCDH's Thursday filing in the Northern Distric...,-0.9920,Bad News,0.4019,Good News
4,OpenAI,"OpenAI unveils latest AI model, customizable G...",OpenAI has unveiled a series of AI tool update...,0.9888,Good News,0.8934,Good News
5,OpenAI,500+ OpenAI employees threaten to quit over CE...,Hundreds of OpenAI staffers are calling for th...,-0.7044,Bad News,-0.9393,Bad News
6,"Facebook, Instagram",Facebook and Instagram users in Europe can now...,Positive news: Meta announced that European us...,0.9753,Good News,0.6705,Good News
7,"Microsoft, OpenAI",Microsoft CEO doesn’t dismiss possibility of S...,Positive news: Sam Altman's return to OpenAI i...,0.9497,Good News,0.3400,Good News
8,OpenAI,OpenAI drama continues: Sam Altman may be mull...,Positive news: Sam Altman is back at OpenAI an...,0.9497,Good News,0.3400,Good News
9,X,Ad execs encourage X CEO Linda Yaccarino to qu...,A groundswell of advertising executives have u...,-0.4755,Bad News,-0.6124,Bad News


In [21]:
# Load
df.to_csv('loaded_data.csv', mode='a', header=False, index=False)

In [26]:
timestamp_format = '%Y-%h-%d-%H:%M:%S'          # Year-Monthname-Day-Hour-Minute-Second 
now = datetime.now()                            # get current timestamp 
timestamp = now.strftime(timestamp_format) 
with open("project_log.txt", "a") as f: 
    f.write(timestamp + ': ' + "Data appended " + '\n') 